In [ ]:
import imblearn
import sklearn

import utils

import pychemauth
from pychemauth.eda.explore import InspectData
from pychemauth.eda.screen import JSBinary
from pychemauth.preprocessing.scaling import CorrectedScaler
from pychemauth.preprocessing.missing import LOD
from pychemauth.classifier.simca import SIMCA_Authenticator
from pychemauth.preprocessing.feature_selection import CollinearFeatureSelector
from pychemauth.manifold.elliptic import EllipticManifold_Authenticator
from pychemauth.analysis.compare import Compare
from pychemauth.classifier import osr

from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.tree import DecisionTreeClassifier
from sklearn.inspection import permutation_importance

import seaborn as sns
import numpy as np
import pandas as pd
import missingno as msno

import matplotlib.pyplot as plt
%matplotlib inline

import watermark
%load_ext watermark

%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
def save_results(results, filename):
    pickle.dump(results, open(filename, 'wb'), protocol=4)

In [ ]:
%watermark -t -m -v --iversions

# Load Data

In [ ]:
use_huggingface = True

if use_huggingface: # Load data from HF
    from datasets import load_dataset

    ds_hf = load_dataset("mahynski/slovenian-site-garlic", token="hf_*",
                        data_files="train.csv")['train'].to_pandas()

    lod_hf = load_dataset("mahynski/slovenian-site-garlic", token="hf_*",
                        data_files="lod.csv")['train'].to_pandas()

    lod_hf['max lod'] = lod_hf[[c for c in lod_hf.columns if 'LOD' in c]].max(axis=1)
    lod = lod_hf['max lod'].set_axis(lod_hf['SITE'])

    X = ds_hf.drop(['Sample type', 'Year', 'Lab code', 'Short LAB code', 'Year of harvesting', 'Country'], axis=1)
    analytes_to_keep = ((X[lod.index] < lod).sum() / X.shape[0] ) <= 0.3
    analytes_to_keep = [a[0] for a in list(analytes_to_keep.items()) if a[1]] 

    X['label'] = X['Origin']
    X.drop('Origin', axis=1, inplace=True)

    X = X[X['label'] != 'Test SLO']
    X = X.loc[:, analytes_to_keep+['label']]
    lod = lod[analytes_to_keep]
else: # Load data from local source
    M_, lod_ = utils.get_data(sheet_name='Cesen_osnova', col_start=7, row_start=17, loc='../raw/DB_UVHVVR.xlsx')

    # We have pre-determined that we wish to drop all analytes with > 30% below LOD.
    analytes_to_keep = ((M_[lod_.index] < lod_).sum() / M_.shape[0] ) <= 0.3
    analytes_to_keep = [a[0] for a in list(analytes_to_keep.items()) if a[1]]

    X = M_.loc[:,analytes_to_keep]
    X['label'] = M_['Poreklo']
    lod = lod_[analytes_to_keep]

    # Let's also drop all the 'Test SLO'
    X = X[X['label'] != 'Test SLO']

_ = msno.matrix(X, labels=True)

# EDA

In [ ]:
selected_features, abroad_cluster_id_to_feature_ids, fig = InspectData.cluster_collinear(
    np.asarray(X[X['label'] != 'Authentic SLO'].drop('label', axis=1).values, dtype=np.float64), 
    feature_names=X[X['label'] != 'Authentic SLO'].drop('label', axis=1).columns, 
    figsize=(12,8),
    t=0.6
)

In [ ]:
selected_features, slo_cluster_id_to_feature_ids, fig = InspectData.cluster_collinear(
    np.asarray(X[X['label'] == 'Authentic SLO'].drop('label', axis=1).values, dtype=np.float64), 
    feature_names=X[X['label'] == 'Authentic SLO'].drop('label', axis=1).columns, 
    figsize=(10,4),
    t=0.6,
    highlight=False,
    figname='garlic_correlations.png'
)

# Clean

In [ ]:
X

In [ ]:
# Also consider dataset composed of ln(X/lod)
X_ln = X.copy()
for te_ in lod.index[4:]:
    X_ln[te_] = X_ln[te_].apply(lambda x: np.log(float(x / lod[te_])))
    
X_auth = X_ln[X_ln['label'] == 'Authentic SLO'].copy()
Outlier = utils.flag_outliers(X_auth, lod)

X_auth_orig = X[X['label'] == 'Authentic SLO'].copy()
X_auth_orig = X_auth_orig[Outlier.drop('label', axis=1).sum(axis=1) < 3]

In [ ]:
X_abroad_orig = X[X['label'] != 'Authentic SLO'].copy()

X_tot = pd.concat((X_auth_orig, X_abroad_orig), axis=0)
X_tot

# JSD

In [ ]:
from pychemauth.eda.explore import InspectData

InspectData.pairplot(X_tot, vars=['Sr', 'Cd', '34S', 'Na', '18O'], hue="label", diag_kind="kde")

In [ ]:
def plot_distribution(X, xlabels):
    fig, axes = plt.subplots(
        nrows=((len(X.columns)-1) // 3) + int(len(X.columns)-1 - ((len(X.columns)-1) // 3) * 3 > 0), 
        ncols=3, 
        figsize=(12,5)
    )

    for i, (site_, ax, label_) in enumerate(zip(X.drop('label', axis=1).columns, axes.ravel(), xlabels)):
        sns.histplot(
            X,
            x=site_,
            hue='label',
            kde=True,
            ax=ax
        )
        ticks_ = ax.get_xticks()[1:-1]
        ax.set_xticks(ticks_, labels=[str('%.0f'%a) for a in ticks_], fontsize=20, rotation=45)
        ax.set_xlabel(label_, fontsize=20)
        
        ticks_ = ax.get_yticks()
        ax.set_yticks(ticks_, labels=[str('%.0f'%a) for a in ticks_], fontsize=20, rotation=0)
        
        if i > 0:
            ax.set_ylabel('')
        else:
            ax.set_ylabel('Count', fontsize=20)
            
    return axes

In [ ]:
axes = plot_distribution(
    X_tot[['Sr', 'Na', '18O', 'label']], 
    xlabels=[r'${\rm Sr}~(\mu g/g)$', r'${\rm Na}~(\mu g/g)$', r'$\delta{}^{18}{\rm O}~({}^{\circ}/{}_{\circ\circ})$']
)

axes[0].axvline(1.173, color='r') # Tree delimiter determined below
axes[1].axvline(50.318, color='r') # Tree delimiter determined below
_ = axes[2].axvline(-2.679, color='r') # Tree delimiter determined below

for ax in axes:
    ax.legend(labels=['Abroad', 'Authentic'], fontsize=18)
    
plt.savefig("fig4b.png", transparent=None, dpi=600, format="png",
        metadata=None, bbox_inches='tight', pad_inches=0.1,
        facecolor='auto', edgecolor='auto', backend=None)

In [ ]:
def plot_jsd(X, vline=11):
    bins = np.arange(1, 101, 10)

    max_value = []
    max_site = []

    for nbins in bins:
        jsb = JSBinary(js_bins=nbins, robust=True)
        jsb.fit(X=X.drop('label', axis=1).values, y=X['label'].values)
        i, j = jsb._JSBinary__enc_.transform(['Authentic SLO'])[0], jsb._JSBinary__enc_.transform(['Abroad'])[0]
        max_value.append(jsb.matrix[i,j])
        max_site.append(jsb.top_features(X.drop('label', axis=1).columns)[i,j])
        
    plt.plot(bins, max_value, '-o')
    plt.title('Authentic vs. Abroad', fontsize=22)
    plt.ylabel('Max JSD', fontsize=20)
    plt.xlabel('Number of Bins', fontsize=20)
    plt.axvline(vline, color='k')
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    
    for i in range(1, len(max_site)):
        plt.text(bins[i]+0.2, max_value[i]-0.07, max_site[i], fontsize=18)
    
    return max_site

In [ ]:
_ = plot_jsd(X_tot, vline=11)

plt.savefig("fig4a.png", transparent=None, dpi=600, format="png",
        metadata=None, bbox_inches='tight', pad_inches=0.1,
        facecolor='auto', edgecolor='auto', backend=None)

# Model

In [ ]:
# Just to guarantee ordering is consistent
SITE = lod.index

In [ ]:
def make_gs(pipeline, param_grid):
    return GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        n_jobs=-1,
        cv=sklearn.model_selection.StratifiedKFold(
            n_splits=10, # 10-fold CV for hyperparameter optimization
            shuffle=True, 
            random_state=0
        ),
        error_score=0,
        refit=True
    )

In [ ]:
def make_pipe(model, use_feature_selector=True):
    steps = [
        ("imputer", LOD( # Impute < LOD
            lod=lod[SITE].values, 
            missing_values=np.nan, 
            seed=42,
            skip_columns=[0, 1, 2, 3] # Stable isotopes are the first 3 features
        ))
    ]
    
    if use_feature_selector:
        steps += [
            ("feature_selector", SelectFromModel( # Must come after LOD_LN imputer
                estimator=CollinearFeatureSelector( # We will replace this estimator in the param_grid below
                    t=0.9,
                    seed=42,
                    minimize_label_entropy=False,
                ),
                threshold=0.5, # 0's are given to features ignored, 1 if they are kept so set the threshold in between
                prefit=False,
            ))
        ]
        
    steps += [
        ("autoscaler", CorrectedScaler( # Data is considered "clean" so we will not use robust estimates
            with_mean=True,
            with_std=True
        )),
        ("model", model),
    ]    
    
    pipeline = imblearn.pipeline.Pipeline(steps=steps)
    
    return pipeline

## DD-SIMCA Model

In [ ]:
# Fit model on entire dataset, but do not use feature selector so that we can see how it chooses to use all of them
dds_param_grid = [{
    'model__n_components':np.arange(1, 10),
    'model__alpha':np.logspace(-5, np.log10(0.5), 20),
}]

dds_pipeline = make_pipe(
    SIMCA_Authenticator(
        n_components=1,
        alpha=0.05,
        scale_x=True,
        style='dd-simca',
        target_class='Authentic SLO',
        use='compliant'
    ),
    use_feature_selector=False, # Let it see all the features
)

dds_gs = make_gs(dds_pipeline, dds_param_grid)

In [ ]:
_ = dds_gs.fit(
    X_tot[SITE].values,
    X_tot['label'].values
)

In [ ]:
dds_gs.score(
    X_tot[SITE].values,
    X_tot['label'].values
)

In [ ]:
dds_gs.best_params_

In [ ]:
ax = dds_gs.best_estimator_.named_steps['model'].model.plot_loadings(
    list(SITE),
    fontsize=18,
    feature_offset=(0.0, 0.01)
)

expl_ = dds_gs.best_estimator_.named_steps['model'].model._DDSIMCA_Model__pca_.explained_variance_ratio_
ax.set_xlabel(f'PC 1 ({"%.4f"%(expl_[0]*100)}%)', fontsize=20)
ax.set_ylabel(f'PC 2 ({"%.4f"%(expl_[1]*100)}%)', fontsize=20)
ticks_ = [-0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.4]
ax.set_xticks(ticks_, [str(a) for a in ticks_], fontsize=18)
ticks_ = [-0.4, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.4]
_ = ax.set_yticks(ticks_, [str(a) for a in ticks_], fontsize=18)

plt.savefig("fig4f_2.png", transparent=None, dpi=600, format="png",
        metadata=None, bbox_inches='tight', pad_inches=0.1,
        facecolor='auto', edgecolor='auto', backend=None)

In [ ]:
preprocessing = imblearn.pipeline.Pipeline(
    steps = dds_gs.best_estimator_.steps[:-1]
)

ax = dds_gs.best_estimator_.named_steps['model'].model.visualize(
    X=preprocessing.transform(X_tot[SITE].values), 
    y=X_tot['label'].values,
    outlier_curve=False
)
ax.set_xlabel('${\\rm ln(1 + h/h_0)}$', fontsize=20)
ax.set_ylabel('${\\rm ln(1 + q/q_0)}$', fontsize=20)
ticks_ = [0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]
ax.set_xticks(ticks_, [str(a) for a in ticks_], fontsize=18)
ticks_ = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
ax.set_yticks(ticks_, [str(a) for a in ticks_], fontsize=18)

_ = ax.legend(labels=['Threshold', 'Abroad = Inlier (9)', 'Abroad = Outlier (44)', 'Authentic = Inlier (72)', 'Authentic = Outlier (1)'], fontsize=16)

plt.savefig("fig4f_1.png", transparent=None, dpi=600, format="png",
        metadata=None, bbox_inches='tight', pad_inches=0.1,
        facecolor='auto', edgecolor='auto', backend=None)

## RF OSR

In [ ]:
# Fit model on entire dataset, but do not use feature selector so that we can see how it chooses to use all of them
osr_param_grid = [{
    'model__clf_kwargs':[{'n_estimators':n, 'random_state':42, 'class_weight':'balanced', 'oob_score':True, 'max_features':None} for n in [20, 50, 100, 150, 200]],
    'model__outlier_kwargs':[
        {'n_estimators': n, 
         'max_samples': 'auto', 
         'contamination': c,
         'max_features': 1.0, 
         'bootstrap':True,
         'n_jobs':-1, 
         'random_state':42} for n in [10, 20, 50, 100] for c in [0.001, 0.01, 0.1, 0.5]
    ],
}]

osr_pipeline = make_pipe(
    osr.OpenSetClassifier(
        clf_model=RandomForestClassifier,
        outlier_model=IsolationForest,
        clf_kwargs={},
        outlier_kwargs={},
        known_classes=['Authentic SLO', 'Abroad'],
        inlier_value=1,
        unknown_class='Unknown',
        score_metric='TEFF',
        clf_style='hard',
        score_using="Authentic SLO"
    ),
    use_feature_selector=False
)

osr_gs = make_gs(osr_pipeline, osr_param_grid)

In [ ]:
_ = osr_gs.fit(
    X_tot[SITE].values,
    X_tot['label'].values
)

In [ ]:
osr_gs.best_score_

In [ ]:
osr_gs.best_params_

In [ ]:
fom = osr_gs.best_estimator_.named_steps['model'].figures_of_merit(
    osr_gs.best_estimator_.predict(X_tot[SITE].values),
    X_tot['label'].values
)

In [ ]:
df_ = fom['CM'].copy()

In [ ]:
df2_ = df_.rename({'Unknown':'OOD = Abroad', 'Authentic SLO':'Authentic'}, axis=1).rename({'Authentic SLO':'Authentic'}, axis=0)

In [ ]:
df2_

In [ ]:
pd.DataFrame(data=[[52+1, 0], [1, 72]], columns=['Abroad','Authentic']).rename({0:'Abroad', 1:'Authentic'}, axis=0)

In [ ]:
fom['I']

In [ ]:
fom['TSNS'], fom['TSPS'], fom['TEFF']

In [ ]:
osr_gs.best_estimator_.named_steps['model'].clf_.oob_score_

In [ ]:
osr_gs.best_estimator_.named_steps['model'].clf_

In [ ]:
# Sr is at the top by a large margin
sorted(zip(SITE, osr_gs.best_estimator_.named_steps['model'].clf_.feature_importances_), key=lambda x: np.abs(x[1]), reverse=True)

In [ ]:
res = permutation_importance(
    osr_gs, 
    X_tot[SITE].values,
    X_tot['label'].values,
    n_repeats=20,
    random_state=42
)

In [ ]:
sort_ = sorted(zip(SITE, res['importances_mean'], res['importances_std']), key=lambda x:np.abs(x[1]), reverse=True)
sort_

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3,5))
ax.plot(
    np.arange(6),
    [np.abs(a[1]) for a in sort_[:6]]
)
_ = plt.xticks(np.arange(6), [a[0] for a in sort_[:5]] + ['...'], fontsize=18)
plt.yticks(fontsize=18)
plt.axhline(0, color='k')
plt.xlabel('SITE Feature', fontsize=20)
plt.ylabel('PFI', fontsize=20)
plt.title('RF OSR (TEFF)', fontsize=22)

plt.savefig("fig4d_1.png", transparent=None, dpi=600, format="png",
        metadata=None, bbox_inches='tight', pad_inches=0.1,
        facecolor='auto', edgecolor='auto', backend=None)

## RF

In [ ]:
# Fit model on entire dataset, include feature selector to mimic dial below
rf_param_grid = [{
    'model__n_estimators':[50, 100, 150, 200],
    'model__max_features':['sqrt', None],
    'feature_selector__estimator': [
        CollinearFeatureSelector(t=0.0, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.2, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.4, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.6, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.8, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=1.0, seed=42, minimize_label_entropy=False),
    ]
}]

rf_pipeline = make_pipe(
    RandomForestClassifier(
        n_estimators=100, 
        random_state=42, 
        oob_score=True, 
        class_weight='balanced',
        max_features=None
    ),
)

rf_gs = make_gs(rf_pipeline, rf_param_grid)

In [ ]:
_ = rf_gs.fit(
    X_tot[SITE].values,
    X_tot['label'].values
)

In [ ]:
rf_gs.score(
    X_tot[SITE].values,
    X_tot['label'].values
)

In [ ]:
rf_gs.best_params_

In [ ]:
# Choose to include everything - equivalent to not using feature selector at all
rf_gs.best_estimator_.named_steps['feature_selector'].get_support()

In [ ]:
sorted(zip(SITE, rf_gs.best_estimator_.named_steps['model'].feature_importances_), key=lambda x: np.abs(x[1]), reverse=True)

In [ ]:
res = permutation_importance(
    rf_gs, 
    X_tot[SITE].values,
    X_tot['label'].values,
    n_repeats=50,
    random_state=42
)

In [ ]:
sort_ = sorted(zip(SITE, res['importances_mean'], res['importances_std']), key=lambda x:np.abs(x[1]), reverse=True)
sort_

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3,5))
ax.plot(
    np.arange(6),
    [np.abs(a[1]) for a in sort_[:6]]
)
_ = plt.xticks(np.arange(6), [a[0] for a in sort_[:5]] + ['...'], fontsize=12)
plt.yticks(fontsize=12)
plt.axhline(0, color='k')
plt.xlabel('SITE Feature', fontsize=14)
plt.ylabel('PFI', fontsize=14)
plt.title('Random Forest', fontsize=16)

## Decision Tree Stump

In [ ]:
# Fit model on entire dataset, but do not use feature selector so that we can see how it chooses to use all of them
dt_param_grid = [{
    'model__max_depth':[1], # Force a stump
}]

dt_pipeline = make_pipe(
    DecisionTreeClassifier(
        max_depth=1,
        random_state=42,
        class_weight='balanced'
    ),
    use_feature_selector=False, # Let it see all the features
)

dt_gs = make_gs(dt_pipeline, dt_param_grid)

In [ ]:
_ = dt_gs.fit(
    X_tot[SITE].values,
    X_tot['label'].values
)

In [ ]:
dt_gs.score(
    X_tot[SITE].values,
    X_tot['label'].values
)

In [ ]:
dt_gs

In [ ]:
from sklearn.tree import plot_tree

fig = plt.figure(figsize=(8,4))
artists = plot_tree(
    dt_gs.best_estimator_.named_steps['model'],
    feature_names=list(SITE),
    class_names=dt_gs.best_estimator_.classes_.tolist(),
    label='root',
    filled=True,
    proportion=False,
    impurity=False,
    ax=fig.gca(),
    fontsize=22,
    rounded=True,
    precision=3,
)
plt.tight_layout()

# De-scale units
d = {a[0]: (a[1], a[2]) for a in list(zip(SITE, dt_gs.best_estimator_.named_steps['autoscaler']._CorrectedScaler__mean_, dt_gs.best_estimator_.named_steps['autoscaler']._CorrectedScaler__std_))}

artists[0].set_text('Sr <= {}\nsamples = 126'.format('%.3f'%(-0.382*d['Sr'][1]+d['Sr'][0])))
artists[1].set_text('70\nAuthentic')
artists[2].set_text('56\nAbroad')

In [ ]:
def train_stump(site='Na'):
    idx = np.where(SITE == site)[0][0]
    
    steps = []
    
    if idx >= 4:
        # Only impute for trace elements
        steps += [("imputer", LOD( # Impute < LOD
            lod=[lod[site]], 
            missing_values=np.nan, 
            seed=42,
        ))]
        
    steps += [("model", DecisionTreeClassifier(
        max_depth=1,
        random_state=42,
        class_weight='balanced'
    ))]
    
    pipeline = imblearn.pipeline.Pipeline(steps=steps)
    
    _ = pipeline.fit(
        X_tot[site].values.reshape(-1,1), 
        X_tot['label'].values
    )
    
    fig = plt.figure(figsize=(6,4))
    artists = plot_tree(
        pipeline.named_steps['model'],
        feature_names=[site],
        class_names=pipeline.named_steps['model'].classes_.tolist(),
        label='root',
        filled=True,
        proportion=False,
        impurity=False,
        ax=fig.gca(),
        fontsize=22,
        rounded=True,
        precision=3,
    )
    plt.tight_layout()
    
    def fix_naming(string):
        s = string.split('\n')
        if len(s) == 4:
            s = [s[0], s[1]]
        else:
            s = [s[0], s[2]]

        if 'Authentic SLO' == s[-1]:
            s[-1] = 'Authentic'
        return s
    
    for a in artists:
        a.set_text('\n'.join(fix_naming(a.get_text())))
    
    return pipeline, pipeline.score(X_tot[site].values.reshape(-1,1),  X_tot['label'].values)

In [ ]:
p_, score_ = train_stump('Sr')
plt.savefig("fig4e_1.png", transparent=None, dpi=600, format="png",
        metadata=None, bbox_inches='tight', pad_inches=0.1,
        facecolor='auto', edgecolor='auto', backend=None)
score_

In [ ]:
p_, score_ = train_stump('Na')
plt.savefig("fig4e_2.png", transparent=None, dpi=600, format="png",
        metadata=None, bbox_inches='tight', pad_inches=0.1,
        facecolor='auto', edgecolor='auto', backend=None)
score_

In [ ]:
p_, score_ = train_stump('18O')
score_

## Comparison

In [ ]:
# RF

rf_param_grid = [{
    'model__n_estimators':[50, 100, 150, 200],
    'model__max_features':['sqrt', None],
    'feature_selector__estimator': [
        CollinearFeatureSelector(t=0.0, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.2, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.4, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.6, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.8, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=1.0, seed=42, minimize_label_entropy=False),
    ]
}]

rf_pipeline = make_pipe(
    RandomForestClassifier(
        n_estimators=100, 
        random_state=42, 
        oob_score=True, 
        class_weight='balanced',
        max_features=None
    ),
)

rf_gs = make_gs(rf_pipeline, rf_param_grid)

In [ ]:
# DD-SIMCA

dds_param_grid = [{
    'model__n_components':np.arange(1, 10),
    'model__alpha':np.logspace(-5, np.log10(0.5), 10),
    'feature_selector__estimator': [
        CollinearFeatureSelector(t=0.0, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.2, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.4, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.6, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.8, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=1.0, seed=42, minimize_label_entropy=False),
    ]
}]

dds_pipeline = make_pipe(
    SIMCA_Authenticator(
        n_components=1,
        alpha=0.05,
        scale_x=True,
        style='dd-simca',
        target_class='Authentic SLO',
        use='compliant'
    ),
)

dds_gs = make_gs(dds_pipeline, dds_param_grid)

In [ ]:
# DD-SIMCA - accuracy

ddsacc_param_grid = [{
    'model__n_components':np.arange(1, 10),
    'model__alpha':np.logspace(-5, np.log10(0.5), 10),
    'feature_selector__estimator': [
        CollinearFeatureSelector(t=0.0, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.2, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.4, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.6, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.8, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=1.0, seed=42, minimize_label_entropy=False),
    ]
}]

ddsacc_pipeline = make_pipe(
    SIMCA_Authenticator(
        n_components=1,
        alpha=0.05,
        scale_x=True,
        style='dd-simca',
        target_class='Authentic SLO',
        use='acc'
    ),
)

ddsacc_gs = make_gs(ddsacc_pipeline, ddsacc_param_grid)

In [ ]:
# UNEQ

uneq_param_grid = [{
    'model__alpha':np.logspace(-5, np.log10(0.5), 10),
    'feature_selector__estimator': [
        CollinearFeatureSelector(t=0.0, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.2, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.4, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.6, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.8, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=1.0, seed=42, minimize_label_entropy=False),
    ]
}]

uneq_pipeline = make_pipe(
    EllipticManifold_Authenticator(
        alpha=0.05,
        robust=True, 
        center='score',
        target_class='Authentic SLO',
        use='compliant'
    ),
)

uneq_gs = make_gs(uneq_pipeline, uneq_param_grid)

In [ ]:
# UNEQ - accuracy

uneqacc_param_grid = [{
    'model__alpha':np.logspace(-5, np.log10(0.5), 10),
    'feature_selector__estimator': [
        CollinearFeatureSelector(t=0.0, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.2, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.4, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.6, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.8, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=1.0, seed=42, minimize_label_entropy=False),
    ]
}]

uneqacc_pipeline = make_pipe(
    EllipticManifold_Authenticator(
        alpha=0.05,
        robust=True, 
        center='score',
        target_class='Authentic SLO',
        use='acc'
    ),
)

uneqacc_gs = make_gs(uneqacc_pipeline, uneqacc_param_grid)

In [ ]:
# DT Stump

dt_param_grid = [{
    'model__max_depth':[1], # Force a stump
    'feature_selector__estimator': [
        CollinearFeatureSelector(t=0.0, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.2, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.4, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.6, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.8, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=1.0, seed=42, minimize_label_entropy=False),
    ]
}]

dt_pipeline = make_pipe(
    DecisionTreeClassifier(
        max_depth=1,
        random_state=42,
        class_weight='balanced'
    ),
)

dt_gs = make_gs(dt_pipeline, dt_param_grid)

In [ ]:
# DT depth 2

dt2_param_grid = [{
    'model__max_depth':[2], 
    'feature_selector__estimator': [
        CollinearFeatureSelector(t=0.0, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.2, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.4, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.6, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.8, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=1.0, seed=42, minimize_label_entropy=False),
    ]
}]

dt2_pipeline = make_pipe(
    DecisionTreeClassifier(
        max_depth=2,
        random_state=42,
        class_weight='balanced'
    ),
)

dt2_gs = make_gs(dt2_pipeline, dt2_param_grid)

In [ ]:
# DT depth 3

dt3_param_grid = [{
    'model__max_depth':[3], 
    'feature_selector__estimator': [
        CollinearFeatureSelector(t=0.0, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.2, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.4, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.6, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.8, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=1.0, seed=42, minimize_label_entropy=False),
    ]
}]

dt3_pipeline = make_pipe(
    DecisionTreeClassifier(
        max_depth=3,
        random_state=42,
        class_weight='balanced'
    ),
)

dt3_gs = make_gs(dt3_pipeline, dt3_param_grid)

In [ ]:
# RF OSR

osr_param_grid = [{
    'model__clf_kwargs':[{'n_estimators':n, 'random_state':42, 'class_weight':'balanced', 'max_features':None} for n in [20, 50, 100, 150, 200]],
    'model__outlier_kwargs':[
        {'n_estimators': n, 
         'max_samples': 'auto', 
         'contamination': c,
         'max_features': 1.0, 
         'bootstrap':True,
         'n_jobs':-1, 
         'random_state':42} for n in [10, 20, 50, 100] for c in [0.001, 0.01, 0.1, 0.5]
    ],
    'feature_selector__estimator': [
        CollinearFeatureSelector(t=0.0, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.2, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.4, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.6, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.8, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=1.0, seed=42, minimize_label_entropy=False),
    ]
}]

osr_pipeline = make_pipe(
    osr.OpenSetClassifier(
        clf_model=RandomForestClassifier,
        outlier_model=IsolationForest,
        clf_kwargs={},
        outlier_kwargs={},
        known_classes=['Authentic SLO', 'Abroad'],
        inlier_value=1,
        unknown_class='Unknown',
        score_metric='TEFF',
        clf_style='hard',
        score_using="Authentic SLO"
    ),
)

osr_gs = make_gs(osr_pipeline, osr_param_grid)

In [ ]:
# RF OSR - accuracy

osracc_param_grid = [{
    'model__clf_kwargs':[{'n_estimators':n, 'random_state':42, 'class_weight':'balanced', 'max_features':None} for n in [20, 50, 100, 150, 200]],
    'model__outlier_kwargs':[
        {'n_estimators': n, 
         'max_samples': 'auto', 
         'contamination': c,
         'max_features': 1.0, 
         'bootstrap':True,
         'n_jobs':-1, 
         'random_state':42} for n in [10, 20, 50, 100] for c in [0.001, 0.01, 0.1, 0.5]
    ],
    'feature_selector__estimator': [
        CollinearFeatureSelector(t=0.0, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.2, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.4, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.6, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=0.8, seed=42, minimize_label_entropy=False),
        CollinearFeatureSelector(t=1.0, seed=42, minimize_label_entropy=False),
    ]
}]

osracc_pipeline = make_pipe(
    osr.OpenSetClassifier(
        clf_model=RandomForestClassifier,
        outlier_model=IsolationForest,
        clf_kwargs={},
        outlier_kwargs={},
        known_classes=['Authentic SLO', 'Abroad'],
        inlier_value=1,
        unknown_class='Unknown',
        score_metric='ACC',
        clf_style='hard',
        score_using="Authentic SLO"
    ),
)

osracc_gs = make_gs(osracc_pipeline, osracc_param_grid)

In [ ]:
rerun = False

if rerun:
    # 10x10 repeated (nested) CV
    scores_rf, scores_dds, scores_uneq, scores_dt, scores_dt2, scores_dt3, scores_osr, scores_ddsacc, scores_uneqacc, scores_osracc = Compare.repeated_kfold(
        [
            rf_gs,
            dds_gs,
            uneq_gs,
            dt_gs,
            dt2_gs,
            dt3_gs,
            osr_gs,
            ddsacc_gs,
            uneqacc_gs,
            osracc_gs
        ],
        X_tot[SITE].values,
        X_tot['label'].values,
        n_repeats=10,
        k=10,
        random_state=0
    )
    
    save_results({
            'Compliant DD-SIMCA (TEFF)': scores_dds,
            'UNEQ (TEFF)': scores_uneq,
            'RF OSR (TEFF)': scores_osr,
            'DT Stump (ACC)': scores_dt,
            'DT Depth 2 (ACC)': scores_dt2,
            'DT Depth 3 (ACC)': scores_dt3,
            'Random Forest (ACC)': scores_rf,
            'Compliant DD-SIMCA (ACC)': scores_ddsacc,
            'UNEQ (ACC)': scores_uneqacc,
            'RF OSR (ACC)': scores_osracc,
        },
        filename='garlic.pkl',
    )
else:
    # Load previously saved results
    scores_ = pickle.load(open('garlic.pkl', 'rb'))

    scores_dds = scores_['Compliant DD-SIMCA (TEFF)']
    scores_osr = scores_['RF OSR (TEFF)']
    scores_uneq = scores_['UNEQ (TEFF)']
    scores_dt = scores_['DT Stump (ACC)']
    scores_dt2 = scores_['DT Depth 2 (ACC)']
    scores_dt3 = scores_['DT Depth 3 (ACC)']
    scores_rf = scores_['Random Forest (ACC)']
    scores_ddsacc = scores_['Compliant DD-SIMCA (ACC)']
    scores_uneqacc = scores_['UNEQ (ACC)']
    scores_osracc = scores_['RF OSR (ACC)']

In [ ]:
ax = Compare.visualize(
    {
        'Compliant DD-SIMCA (TEFF)': scores_dds,
        'UNEQ (TEFF)': scores_uneq,
        'RF OSR (TEFF)': scores_osr,
        'DT Stump (ACC)': scores_dt,
        'DT Depth 2 (ACC)': scores_dt2,
        'DT Depth 3 (ACC)': scores_dt3,
        'Random Forest (ACC)': scores_rf,
        'Compliant DD-SIMCA (ACC)': scores_ddsacc,
        'UNEQ (ACC)': scores_uneqacc,
        'RF OSR (ACC)': scores_osracc,
    },
    n_repeats=10,
    alpha=0.05,
    ignore=0,
    cmap='plasma'
)

_ = ax.set_xticklabels(
    ["%.1f" % t for t in np.linspace(0.0, 0.9, 10)],
    fontsize=18
)
_ = ax.set_title("Score +/- " + r"$\sigma$", fontsize=22)
l = ax.legend(loc="best", bbox_to_anchor=(0.85, 0.5, 0.5, 0.5), fontsize=18)

for text in l.get_texts():
    if 'RF OSR' in text.get_text():
        text.set_color('blue')
    else:
        text.set_color("green")
        
plt.savefig("fig4c.png", transparent=None, dpi=600, format="png",
        metadata=None, bbox_inches='tight', pad_inches=0.1,
        facecolor='auto', edgecolor='auto', backend=None)

In [ ]:
np.mean(scores_dds[scores_dds>0]), np.std(scores_dds[scores_dds>0])

In [ ]:
# # Create models for HuggingFace
# for name, pipe in [
#     ('random-forest[acc]', rf_gs),
#     ('compliant-dd-simca[teff]', dds_gs),
#     ('uneq[teff]', uneq_gs),
#     ('dt-stump[acc]', dt_gs),
#     ('dt-depth-2[acc]', dt2_gs),
#     ('dt-depth-3[acc]', dt3_gs),
#     ('random-forest-osr[teff]', osr_gs),
#     ('compliant-dd-simca[acc]', ddsacc_gs),
#     ('uneq[acc]', uneqacc_gs),
#     ('random-forest-osr[acc]', osracc_gs)
# ]:
#     _ = pipe.fit(
#         X_tot[SITE].values,
#         X_tot['label'].values
#     )
    
#     pickle.dump(pipe, open('model-'+name+'.pkl', 'wb'), protocol=4)